In [ ]:
import minRLHF
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    AutoModelForTokenClassification
)

In [1]:
from minRLHF.environment import Environment
import random
from transformers.pipelines import pipeline

reward_model = pipeline(
    "text-classification",
    model='bhadresh-savani/distilbert-base-uncased-emotion', 
    return_all_scores=True
)

class MyEnv(Environment):
    def get_input_prompt(self) -> str:
        return random.choice([
            'I went for a walk one day and',
            'A long time ago, in a galaxy far far away',
            'Oops! I'
        ])
        
    def score_generation(self, text: str) -> float:
        return reward_model(text)[0][0]['score']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')
critic = AutoModelForTokenClassification.from_pretrained('gpt2')

# Instantiate envrionment
env = MyEnv(tokenizer, batch_size=4)

# Create PPO trainer
ppo_trainer = PPOTrainer(
    actor=model,
    critic=critic,
    env=env
)

ppo_trainer.train()